In [1]:
# %matplotlib inline
%matplotlib qt
import hyperspy.api as hs
import numpy as np
import lmfit as lm
import matplotlib.pyplot as plt
import espm.spectrum_fitting as sf
from pathlib import Path
from espm.estimators import SmoothNMF

# Guide to this notebook

Every time the symbol ⚠️ appears, a user input is required in the cell below.

If a cell made a few windows pop up, please kill the windows once you're done. Otherwise every new plot will be displayed in top of the other (which may slow your computer down). 

Overview of the different steps :
- I. Load the data either a spectrum (go to step III.) or a spectrum image
- II. Select an area of interest in your spectrum image.
- III. Creates an energy scale (x) from your data for the fitting procedure.
- IV. Selects the regions of the spectrum were there are no peaks for background fitting.
- V. Creates the required objects (partial x and y) for the background fitting procedure.
- VI. Fitting the background. If this is not satisfactory go back to IV.

## I. Load spectrum & energy scale

This cell will start a gui, that will let you select the dataset you want to analyse. In this notebook, only single spectra are treated, the data are thus automatically summed and cropped.

⚠️

In [2]:
spectrum = hs.load().mean().isig[0.2:]

# spectrum.set_signal_type('EDS_espm')

offset = spectrum.axes_manager[0].offset
scale = spectrum.axes_manager[0].scale
size = spectrum.axes_manager[0].size

x = np.linspace(offset,size*scale+offset,num = size)

/home/adrien/anaconda3/envs/espm_dev/lib/python3.11/site-packages/hyperspy/misc/utils.py:471: VisibleDeprecationWarning: Use of the `binned` attribute in metadata is going to be deprecated in v2.0. Set the `axis.is_binned` attribute instead. 
  warnings.warn(
/home/adrien/anaconda3/envs/espm_dev/lib/python3.11/site-packages/hyperspy/io.py:572: VisibleDeprecationWarning: Loading old file version. The binned attribute has been moved from metadata.Signal to axis.is_binned. Setting this attribute for all signal axes instead.
  warnings.warn('Loading old file version. The binned attribute '


## Setting the relevant metadata

This cell is important later during the fitting of the characteristic peaks. Check the documentation at : https://espm.readthedocs.io/en/latest/ for details about thos parameters

⚠️

In [3]:
spectrum.set_analysis_parameters(beam_energy = 200,
                                 azimuth_angle = 0.0,
                                 elevation_angle = 22.0,
                                 tilt_stage = 0.0,
                                 elements = ['Sr', 'Ti', 'O'],
                                 thickness = 200e-7,
                                 density = 3.5,
                                 detector_type = "SDD_efficiency.txt",
                                 width_slope = 0.01,
                                 width_intercept = 0.065,
                                 xray_db = "200keV_xrays.json")

In [3]:
spectrum.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 35.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       ├── take_off_angle = 35.0
│       │       ├── type = SDD_efficiency.txt
│       │       ├── width_intercept = 0.065
│       │       └── width_slope = 0.01
│       ├── Stage
│       │   └── tilt_alpha = 0.0
│       ├── acquisition_mode = STEM
│       ├── beam_energy = 200.0
│       └── microscope = Microscope TecnaiOsiris 200 kV D658 AnalyticalTwin
├── General
│   ├── FileIO
│   │   └── 0
│   │       ├── hyperspy_version = 1.7.6
│   │       ├── io_plugin = hyperspy.io_plugins.hspy
│   │       ├── operation = load
│   │       └── timestamp = 2024-04-04T13:26:39.852818+02:00
│   ├── date = 14.10.2014
│   └── title = Core shell
├── Sample
│   ├── density = 3.5
│   ├── elements = ['Fe', 'Pt', 'O', 'Si', 'C', 'Cu']
│   ├── thickness = 2e-05
│   └── xray_lines = array(['Fe_Ka', 'Pt_La'], dtype='<U5')
├── Signal
│   ├── signal_origin = 
│   └── signal_type = EDS_espm
└── xray_db = 200keV_xrays.json

# II. Select areas for bckgd fitting

⚠️ Input in the ``span_number`` the number of selection areas you wish to use. Once you execute the cell, a window will pop with a few green areas and a red spectrum. The green areas correspond to the regions were the background will be fitted. You can, drag, enlarge and reduce these green areas.

Note : Click on one of the green area before the click and drag operation so that only one of the area is selected and so avoid to drag them all together.

In [21]:
span_number = 4

def selection_areas(number,spectrum,scale) :
    spectrum.plot()
    size = spectrum.axes_manager[0].size //(2*number)
    roi_list = []
    for i in range(number) :
        roi_list.append(hs.roi.SpanROI(offset+ size*scale + 2*i*size*scale, 2*size*scale + offset+ 2*i*size*scale))
        roi_list[-1].interactive(spectrum)
    return roi_list

spans = selection_areas(span_number,spectrum,scale)

Once you are satisfied with the selected areas, save their positions using the cell below

In [22]:
list_energies = [[roi.left,roi.right] for roi in spans]

# III. Fitting the continuum

⚠️ Input the sample parameters below and execute the cell to fit

In [6]:
thickness = 2e-5
density = 3.5
take_off_angle = 35
elements_dict = {"Fe" : 0.0194, "C" : 0.8904, "Pt" : 0.0051, 'O' : 0.03797, 'Si' : 0.00850 , "Cu" : 0.03846}
detector = "SDD_efficiency.txt"

#################################################################################################
# For a custom detector, uncomment below and replace "SDD_efficiency.txt" with the dictionnary. #
#################################################################################################

# {"detection" : {
#     "thickness" : 450e-4,
#     "elements_dict" : {
#         "Si" : 1.0
#     }
# },"layer" : {
#     "thickness" : 10e-7,
#     "elements_dict" : {
#         "Si" : 0.33,
#         "O" : 0.66,
#         "Al" : 1.0
#     }
# }}

part_x, part_y, sum_boola = sf.make_partial_xy(list_energies,spectrum,x)

example = {
    "E0" : 200,
    "b0" : 1.0,
    "b1" : 1.0,
    "params_dict" : {
    "Det" : detector,
    'Abs' : {
        "thickness" : thickness,
        "toa" : take_off_angle,
        "density" : density}
    },
    "elements_dict" : elements_dict
}

pars = sf.ndict_to_params(example)

#################################################################################################
# You can uncomment the lines below if you want to add constraints to the absorption parameters #
#################################################################################################

pars["params_dict__Abs__thickness"].vary = False
# pars["params_dict__Abs__thickness"].max = 5000e-7
# pars["params_dict__Det__layer__thickness"].vary = False
pars["params_dict__Abs__toa"].vary = False
pars["params_dict__Abs__density"].vary = False
pars["E0"].vary = False

out = lm.minimize(sf.residual, pars, args=(part_x,), kws={'data': part_y})
print(lm.fit_report(out))

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 16
    # data points      = 289
    # variables        = 2
    chi-square         = 0.00301149
    reduced chi-square = 1.0493e-05
    Akaike info crit   = -3311.33528
    Bayesian info crit = -3304.00243
[[Variables]]
    E0:                           200 (fixed)
    b0:                           1.17839908 +/- 0.08045653 (6.83%) (init = 1)
    b1:                           0.14987934 +/- 0.00148165 (0.99%) (init = 1)
    params_dict__Abs__thickness:  2e-05 (fixed)
    params_dict__Abs__toa:        35 (fixed)
    params_dict__Abs__density:    3.5 (fixed)
    elements_dict__Fe:            0.0194 (fixed)
    elements_dict__C:             0.8904 (fixed)
    elements_dict__Pt:            0.0051 (fixed)
    elements_dict__O:             0.03797 (fixed)
    elements_dict__Si:            0.0085 (fixed)
    elements_dict__Cu:            0.03846 (fixed)
[[Correlations]] (unreported correlations are < 0.100)
    C(b0, 

### Plotting the results

The red curves corresponds to the background model, the black one to the data, and the grey area correspond to the selected green areas

In [7]:
y = spectrum.data
bkgd = sf.residual(out.params,x)
plt.plot(x,y,"ko-",markersize=1.5,label="exp",markevery=10)
plt.fill_between(x,0,y,where=sum_boola,label="fit windows",color="0.8")
plt.xlabel("Energy (keV)",fontsize=22)
plt.xticks(fontsize=20)
# plt.ylim(0,1)
plt.yticks(fontsize=20)
plt.ylabel("Intensity",fontsize=22)
plt.plot(x,bkgd,"r-",linewidth=1,label="fit")
plt.legend(fontsize=22)

# VI. Quantify the chemical concentrations

## Fitting parameters setup

First remove the fitted background from the complete spectrum and remove the negative values.

In [ ]:
spectrum.data = (spectrum.data - bkgd).clip(min = 0.0)

In this cell we build the characteristic X-ray model to quantify the spectrum.

Using the `reference_elt` keyword argument you can decouple characteristic X-ray lines of single elements to get a better fit. It can be necessary when the absorption is incorrectly calculated or when the cross-sections are not well calculated (e.g. for heavy elements).

The `reference_elt` argument is a dictionary with `{ 'atomic number of the chosen element (int) : energy cutoff (float), .... }`, this way, the intensity of the lines above and below the cutoff are decoupled.

⚠️

In [ ]:
spectrum.build_G(problem_type='no_brstlg',reference_elt={'22' : 3.0})

In this cell we set up the parameters of the fitting algorithm.

For most fitting, you don't need to change those parameters.

In [ ]:
est = SmoothNMF(G=spectrum.model,n_components= 1,max_iter=2000,tol = 1e-8,hspy_comp = False)

## Fitting the spectrum

In [ ]:
est.fit_transform(X = spectrum.data[:,np.newaxis] ,H=np.array([1.0])[:,np.newaxis])

## Printing the concentrations

In relative atomic concentrations

In [ ]:
spectrum.print_concentration_report()

Only printing selected elements. For example Cu is almost always present but not relevant to the quantification. It can be nice to remove it by selecting only the relevant elements.

In [ ]:
spectrum.print_concentration_report(selected_elts=['Sr'])

# IX. Plot the results

⚠️ The popping window will display the bkgd substracted experimental spectrum, the fitted characteristic Xrays model in black and red. Every element composing the model will be displayed with an offset : ``components_offset``. You can change its value below. 

If you want to display the names of the lines of an element that was used for quantification execute the next cells below before kill the window.

In [ ]:
components_offset = -0.1

In [ ]:
linestyles = [":","--","-."]

plt.plot(x,spectrum.data,"ko-",label="exp",markevery = 10)
plt.plot(x,spectrum.G@est.W_,"r",linewidth = 3,label="theo")
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.xlabel("energy (keV)",fontsize = 18)
plt.ylabel("Intensity", fontsize = 18)

for i in range(spectrum.G.shape[1]) :
    ls_string = linestyles[i%len(linestyles)] + "C{}".format(i%9)
    plt.plot(x,spectrum.G[:,i]*est.W_[i]+components_offset,ls_string,label=str(spectrum.metadata.EDS_model.elements[i]),linewidth=3)


plt.legend(fontsize=18)
plt.tight_layout()